In [9]:
import torch
torch.__version__

'1.5.0+cu101'

In [10]:
from utils import load_checkpoint, save_checkpoint, Params
from model.lstm_net import LSTMNet

In [11]:
device = torch.device("cuda:1")
params = Params("./experiment/LSTMNet/wider_or_deeper/256x2/params.json")

model = LSTMNet(params, device).to(device)

In [13]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0154,  0.0505],
        [ 0.0090,  0.0434],
        [ 0.0120, -0.0002],
        ...,
        [ 0.0366,  0.0159],
        [-0.0449,  0.0110],
        [ 0.0590,  0.0122]], device='cuda:1', requires_grad=True)
Parameter containing:
tensor([[ 0.0297,  0.0365,  0.0441,  ..., -0.0237, -0.0191,  0.0419],
        [ 0.0424, -0.0597,  0.0259,  ..., -0.0083,  0.0174, -0.0217],
        [ 0.0260,  0.0245, -0.0067,  ...,  0.0618, -0.0610, -0.0536],
        ...,
        [-0.0185,  0.0041, -0.0043,  ...,  0.0460, -0.0537,  0.0620],
        [-0.0349, -0.0221, -0.0054,  ...,  0.0322, -0.0089, -0.0239],
        [ 0.0052, -0.0502, -0.0245,  ...,  0.0245,  0.0205, -0.0509]],
       device='cuda:1', requires_grad=True)
Parameter containing:
tensor([ 0.0077,  0.0134,  0.0615,  ..., -0.0033,  0.0268, -0.0147],
       device='cuda:1', requires_grad=True)
Parameter containing:
tensor([-0.0267, -0.0167,  0.0139,  ...,  0.0487, -0.0221, -0.0351],
       device='cuda:1', requires_

In [14]:
load_checkpoint("./experiment/LSTMNet/wider_or_deeper/256x2/best.pth.tar", model, cuda_id=1)

parameter is loaded


(LSTMNet(
   (double_lstm): LSTM(2, 256, num_layers=2, batch_first=True)
   (batchnorm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (linear1): Linear(in_features=256, out_features=128, bias=True)
   (selu): SELU()
   (linear2): Linear(in_features=128, out_features=11, bias=True)
 ),
 {'epoch': 272,
  'state_dict': OrderedDict([('double_lstm.weight_ih_l0',
                tensor([[-0.0244, -0.0317],
                        [ 0.9271, -0.9198],
                        [-0.0577, -0.1632],
                        ...,
                        [ 0.1083, -0.2710],
                        [-0.0800,  0.4346],
                        [-0.0218,  0.4427]], device='cuda:1')),
               ('double_lstm.weight_hh_l0',
                tensor([[ 0.1164,  0.0073, -0.1953,  ..., -0.2294,  0.2629, -0.2372],
                        [ 0.4988, -1.0350, -0.0319,  ...,  0.4569, -0.7248,  0.2941],
                        [-0.5385,  0.1931, -0.3764,  ...,  1.1179,  0.24

In [16]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.0244, -0.0317],
        [ 0.9271, -0.9198],
        [-0.0577, -0.1632],
        ...,
        [ 0.1083, -0.2710],
        [-0.0800,  0.4346],
        [-0.0218,  0.4427]], device='cuda:1', requires_grad=True)
Parameter containing:
tensor([[ 0.1164,  0.0073, -0.1953,  ..., -0.2294,  0.2629, -0.2372],
        [ 0.4988, -1.0350, -0.0319,  ...,  0.4569, -0.7248,  0.2941],
        [-0.5385,  0.1931, -0.3764,  ...,  1.1179,  0.2496,  0.6555],
        ...,
        [ 0.1578,  0.0672,  0.0965,  ..., -0.5086,  0.0489,  0.5367],
        [-0.4597,  0.7701, -0.3491,  ..., -0.2878, -0.5818, -0.6886],
        [ 0.0550,  1.0389,  0.9212,  ..., -0.4000, -0.2066,  1.2081]],
       device='cuda:1', requires_grad=True)
Parameter containing:
tensor([-0.8854, -0.6464, -0.5740,  ..., -0.1794, -0.5996, -0.3682],
       device='cuda:1', requires_grad=True)
Parameter containing:
tensor([-0.8604, -0.6065, -0.6118,  ..., -0.1932, -0.6632, -0.3705],
       device='cuda:1', requires_

In [20]:
import _pickle as pickle
import h5py
import numpy as np
import argparse
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from tqdm import trange

data_dir = "./data/RML2016.10a_dict.pkl"
with open(data_dir, "rb") as f:
    data = pickle.load(f, encoding = "iso-8859-1")

mod_list = list({key[0] for key in data.keys()})
SNR_range = range(-12, 12, 2)
feature_dict = {}
logit_dict = {}

parser = argparse.ArgumentParser()
parser.add_argument("--num_sample")
parser.add_argument("--num_type")
args = parser.parse_args()

sample_per_pair = int(args.num_sample)
num_type = int(args.num_type)


for state in trange(1<<11):
    idx_list = []
    label = np.zeros((1,11), dtype=np.int)
    name = ""
    cnt = 0
    for j in range(11):
        if (state == 0): break
        state >>= 1
        if (state & 1):
            cnt += 1
            idx_list.append(j)
            label[0][j] = 1
            name += "_" + mod_list[j]
    if (cnt != num_type): continue
    label = np.repeat(label, sample_per_pair, axis=0)
    for snr in SNR_range:
        if (snr not in feature_dict):
            feature_dict[snr] = []
            logit_dict[snr] = []
        cur_feature_arr = np.zeros((sample_per_pair, 2, 128))
        logit_dict[snr].append(label)
        for idx in idx_list:
            mod = mod_list[idx]
            cur_feature_arr += data[(mod, snr)][:sample_per_pair]
        feature_dict[snr].append(cur_feature_arr)

for snr in SNR_range:
    feature_dict[snr] = np.vstack(feature_dict[snr])
    logit_dict[snr] = np.vstack(logit_dict[snr])
    feature_arr[snr] = feature_arr[snr].reshape((feature_arr[snr].shape[0], -1))
    feature_arr[snr] = normalize(feature_arr[snr], norm="l2")
    feature_arr[snr] = feature_arr[snr].reshape((feature_arr[snr].shape[0], 2, 128))
    feature_arr[snr] = feature_arr[snr].transpose([0, 2, 1])

with open(f"./data/test_data_{num_type}_{sample_per_pair}.h5", "wb") as f:
    pickle.dump((feature_dict, logit_dict), f, protocol=4);
# with h5py.File(f"./data/test_data_{num_type}_{sample_per_pair}.h5", "w") as f:
#     f.create_dataset("logit_dict", data=logit_dict)
#     f.create_dataset('feature_dict', data=feature_dict)

usage: ipykernel_launcher.py [-h] [--num_sample NUM_SAMPLE]
                             [--num_type NUM_TYPE]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1001/jupyter/kernel-ccaa04cb-0de4-4787-b216-59cd286bace4.json


SystemExit: 2

/home/kn/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
